<a href="https://colab.research.google.com/github/ashwinvaswani/Generative-Modelling-of-Images-from-Speech/blob/master/src/TIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pydub
!pip install pytube==9.5.1
!pip install mtcnn
!pip install keras-vggface

     |████████████████████████████████| 2.3MB 39kB/s 


In [0]:
!pip install youtube-dl

     |████████████████████████████████| 1.8MB 2.8MB/s 


In [0]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 
import os
import sys
import moviepy.editor as mp
import pickle

import math
from math import ceil
from math import floor
import time

from pydub import AudioSegment
from pytube import YouTube

################# TEMP ####################
import youtube_dl
########################################### 

import shutil

import IPython.display as ipd
import librosa
import librosa.display

import dlib

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH = './drive/My Drive/TIP/Dataset/'
PATH_TO_MAIN = './drive/My Drive/TIP/'
YT_LINK = 'http://www.youtube.com/watch?v='

In [0]:
column_names = ['YouTube_ID', 'start_segment', 'end_segment', 'X_coordinate', 'Y_coordinate']
train_df = pd.read_csv(PATH + 'avspeech_train.csv',names = column_names)
test_df = pd.read_csv(PATH + 'avspeech_test.csv',names = column_names)

In [0]:
train_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,CJoOwXcjhds,233.266000,239.367000,0.780469,0.670833
1,AvWWVOgaMlk,90.000000,93.566667,0.586719,0.311111
2,Y8HMIm8mdns,171.607767,174.607767,0.505729,0.240741
3,akwvpAiLFk0,144.680000,150.000000,0.698438,0.288889
4,Swss72CHSWg,90.023267,97.297200,0.230729,0.204630


In [0]:
test_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,u5MPyrRJPmc,108.2400,111.240000,0.849219,0.305556
1,H1ulMfj5wRY,112.3200,116.940000,0.112500,0.345833
2,-wuxbgMRIWs,30.0300,36.745044,0.744531,0.211111
3,GNRPRH-E-sI,30.2302,38.171467,0.333594,0.494444
4,VvcwAGkSy2o,240.2000,253.366667,0.491667,0.372222


In [0]:
YT_LINK + train_df.iloc[4]["YouTube_ID"]

'http://www.youtube.com/watch?v=Swss72CHSWg'

In [0]:
# yt = YouTube('www.youtube.com/watch?v=AvWWVOgaMlk')

In [0]:
def dwl_vid(zxt): 
    with youtube_dl.YoutubeDL(ydl_opts) as ydl: 
        ydl.download([zxt]) 

In [0]:
# shutil.rmtree('Images')
# shutil.rmtree('Audio')
# shutil.rmtree('Videos')
shutil.rmtree(PATH_TO_MAIN + 'Face_Images/')
os.mkdir("Images")
os.mkdir("Audio")
os.mkdir("Videos")
os.mkdir(PATH_TO_MAIN + 'Face_Images/')

len_train = train_df.shape[0]
count_unsuccessful = 0
dict_audio = {}
to_remove = []

for i in range(1,150):
  if(True):
    print(i)
    print(YT_LINK + train_df.iloc[i]["YouTube_ID"])
    try:
      # yt = YouTube(YT_LINK + train_df.iloc[i]["YouTube_ID"])
      # stream = yt.streams.first()
      # stream.download(output_path = 'Videos/',filename = train_df.iloc[i]["YouTube_ID"])

      ################# TEMP ####################
      ydl_opts = {'outtmpl':'Videos/'+train_df.iloc[i]["YouTube_ID"]} 
      link = YT_LINK + train_df.iloc[i]["YouTube_ID"]
      dl_input = link.strip()
      dwl_vid(dl_input)
      ###########################################

      cap = cv2.VideoCapture("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mkv') 
      fps = cap.get(cv2.CAP_PROP_FPS)
      
      if fps == 0:
        print("0 fps")
        continue

      count = 0

      dir_name = train_df.iloc[i]["YouTube_ID"]
      
      print("FPS is : "  + str(fps))
      start_cnt = train_df.iloc[i]["start_segment"]
      end_cnt = train_df.iloc[i]["end_segment"]
      
      while(1):

          ret,frame = cap.read()

          if ret == 1:

            count += 1

            (h, w) = frame.shape[:2]

            if (((start_cnt + end_cnt)*fps)/2) - 16 < count < (((start_cnt + end_cnt)*fps)/2) + 16:
              dnnFaceDetector = dlib.cnn_face_detection_model_v1( PATH_TO_MAIN + "Face_detection/mmod_human_face_detector.dat")
              faceRects = dnnFaceDetector(frame, 0)
              print(faceRects)

              if len(faceRects) == 0:
                to_remove.append(dir_name)
                continue
              for faceRect in faceRects:
                  x1 = faceRect.rect.left()
                  y1 = faceRect.rect.top()
                  x2 = faceRect.rect.right()
                  y2 = faceRect.rect.bottom()
                  img = frame[y1:y2, x1:x2]
                  print("About to crop")
                  print(count)
                  img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
                  cv2.imwrite('Images/' +dir_name + '.png', img)
                  cv2.imwrite(PATH_TO_MAIN + 'Face_Images/' +dir_name + '.png', img)
              break


            if count > int(end_cnt*fps):
              break
            k = cv2.waitKey(1)
            if k == ord('q'):
              break

      cap.release()
      cv2.destroyAllWindows()
      time.sleep(60)
    except:
      print("###########################################")
      print("SORRY!!! The video is unavailable")
      print("###########################################")
      time.sleep(120)
      count_unsuccessful += 1
      continue
      
    try:
      clip = mp.VideoFileClip("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mkv').subclip(start_cnt,end_cnt)
      clip.audio.write_audiofile("Audio/" + train_df.iloc[i]["YouTube_ID"] +'.mp3') 
      print("before dict")
      dict_audio[train_df.iloc[i]["YouTube_ID"]] = end_cnt - start_cnt
    except:
      print("###########################################")
      print("Error with audio")  
      print("###########################################")
      continue
    
    

1
http://www.youtube.com/watch?v=AvWWVOgaMlk
[youtube] AvWWVOgaMlk: Downloading webpage
[youtube] AvWWVOgaMlk: Downloading video info webpage


[download] Destination: Videos/AvWWVOgaMlk.f136
[download] 100% of 251.71MiB in 00:50
[download] Destination: Videos/AvWWVOgaMlk.f251
[download] 100% of 15.93MiB in 00:01
[ffmpeg] Merging formats into "Videos/AvWWVOgaMlk.mkv"
Deleting original file Videos/AvWWVOgaMlk.f136 (pass -k to keep)
Deleting original file Videos/AvWWVOgaMlk.f251 (pass -k to keep)
FPS is : 30.0
mmod_rectangles[[(532, 80) (871, 420)]]
About to crop
2738
[MoviePy] Writing audio in Audio/AvWWVOgaMlk.mp3


100%|██████████| 79/79 [00:00<00:00, 466.93it/s]

[MoviePy] Done.
before dict
2
http://www.youtube.com/watch?v=Y8HMIm8mdns
[youtube] Y8HMIm8mdns: Downloading webpage


[youtube] Y8HMIm8mdns: Downloading video info webpage


...

[download] Destination: Videos/uxCc_3ybp1w.f136
[download] 100% of 265.87MiB in 00:29
[download] Destination: Videos/uxCc_3ybp1w.f251
[download] 100% of 23.98MiB in 00:04
[ffmpeg] Merging formats into "Videos/uxCc_3ybp1w.mkv"
Deleting original file Videos/uxCc_3ybp1w.f136 (pass -k to keep)
Deleting original file Videos/uxCc_3ybp1w.f251 (pass -k to keep)
FPS is : 29.97002997002997
mmod_rectangles[[(1030, 391) (1167, 527)]]
About to crop
5459
[MoviePy] Writing audio in Audio/uxCc_3ybp1w.mp3


100%|██████████| 81/81 [00:00<00:00, 406.42it/s]

[MoviePy] Done.
before dict
142
http://www.youtube.com/watch?v=zA_KKsewlN0
[youtube] zA_KKsewlN0: Downloading webpage


[youtube] zA_KKsewlN0: Downloading video info webpage
[download] Destination: Videos/zA_KKsewlN0.f136
[download] 100% of 49.28MiB in 00:07
[download] Destination: Videos/zA_KKsewlN0.f140
[download] 100% of 4.90MiB in 00:00
[ffmpeg] Merging formats into "Videos/zA_KKsewlN0.mp4"
Deleting original file Videos/zA_KKsewlN0.f136 (pass -k to keep)
Deleting original file Videos/zA_KKsewlN0.f140 (pass -k to keep)
0 fps
143
http://www.youtube.com/watch?v=Drw4N5gyE5s
[youtube] Drw4N5gyE5s: Downloading webpage
[youtube] Drw4N5gyE5s: Downloading video info webpage
[download] Destination: Videos/Drw4N5gyE5s.f247
[download] 100% of 79.10MiB in 00:19
[download] Destination: Videos/Drw4N5gyE5s.f251
[download] 100% of 9.94MiB in 00:02
[ffmpeg] Merging formats into "Videos/Drw4N5gyE5s.webm"
Deleting original file Videos/Drw4N5gyE5s.f247 (pass -k to keep)
Deleting original file Videos/Drw4N5gyE5s.f251 (pass -k to keep)
0 fps
144
http://www.youtube.com/watch?v=zjY8yp0Lea4
[youtube] zjY8yp0Lea4: Downloading

[dashsegments] Total fragments: 89
[download] Destination: Videos/raN3iblWVCA.f136
[download] 100% of 107.62MiB in 00:55
[dashsegments] Total fragments: 45
[download] Destination: Videos/raN3iblWVCA.f251
[download] 100% of 6.05MiB in 00:17
[ffmpeg] Merging formats into "Videos/raN3iblWVCA.mkv"
Deleting original file Videos/raN3iblWVCA.f136 (pass -k to keep)
Deleting original file Videos/raN3iblWVCA.f251 (pass -k to keep)
FPS is : 23.976076555023923
mmod_rectangles[[(463, -23) (803, 317)]]
About to crop
2106
###########################################
SORRY!!! The video is unavailable
###########################################
148
http://www.youtube.com/watch?v=QrSD1SiWE-g
[youtube] QrSD1SiWE-g: Downloading webpage
[youtube] QrSD1SiWE-g: Downloading video info webpage
[download] Destination: Videos/QrSD1SiWE-g.f136
[download] 100% of 99.71MiB in 00:07
[download] Destination: Videos/QrSD1SiWE-g.f140
[download] 100% of 4.52MiB in 00:00
[ffmpeg] Merging formats into "Videos/QrSD1SiWE-g.mp

In [58]:
cnt = 0
list_aud = []
list_img = []
for elem in sorted(os.listdir(PATH_TO_MAIN + 'Face_Images/')):
  list_img.append(elem)
  cnt += 1

for elem in sorted(os.listdir('./Audio')):
  list_aud.append(elem)
  cnt += 1

for i in range(len(list_aud)):
  print(list_aud[i],list_img[i])
print(cnt)

-A9gdf3j2xo.mp3 -A9gdf3j2xo.png
2f32XSMYlDk.mp3 2f32XSMYlDk.png
307DK9nGQhw.mp3 307DK9nGQhw.png
3AsPqH3QaQM.mp3 3AsPqH3QaQM.png
3gcWAZSNi2E.mp3 3gcWAZSNi2E.png
447M_WXVZz8.mp3 447M_WXVZz8.png
4dUJVqvs2Qo.mp3 4dUJVqvs2Qo.png
512K2S3De-A.mp3 512K2S3De-A.png
5qy9Ujv9XdM.mp3 5qy9Ujv9XdM.png
6LapKTptu8w.mp3 6LapKTptu8w.png
7DBdAnTuw5c.mp3 7DBdAnTuw5c.png
8BQxktcqJjU.mp3 8BQxktcqJjU.png
8nQBG5hvjpk.mp3 8nQBG5hvjpk.png
9W-s6g88Vhw.mp3 9W-s6g88Vhw.png
AOoqrXx5BNU.mp3 AOoqrXx5BNU.png
AvWWVOgaMlk.mp3 AvWWVOgaMlk.png
BJsd0c6gi5U.mp3 BJsd0c6gi5U.png
BjvtZkHWExY.mp3 BjvtZkHWExY.png
Dtn8xZ3BiGY.mp3 Dtn8xZ3BiGY.png
DuWE-CQDlEk.mp3 DuWE-CQDlEk.png
DxpQmBfA6vM.mp3 DxpQmBfA6vM.png
G7xm-5aDZyg.mp3 G7xm-5aDZyg.png
Gqt1A6O6UTk.mp3 Gqt1A6O6UTk.png
GtM2sM6r3So.mp3 GtM2sM6r3So.png
H2VCPO0isFQ.mp3 H2VCPO0isFQ.png
HoeRIgYg9s4.mp3 HoeRIgYg9s4.png
IrXrbrZWflA.mp3 IrXrbrZWflA.png
JjduaMIoKvI.mp3 JjduaMIoKvI.png
LcyrfLT2tto.mp3 LcyrfLT2tto.png
MZnQ3eZuUAE.mp3 MZnQ3eZuUAE.png
M_HcFLa0b1Q.mp3 M_HcFLa0b1Q.png
QYnLgIsR

In [0]:
# Preparing y_train for encoder and x_train for decoder

In [0]:
def get_embeddings(filenames):

  faces = []
  for f in filenames:
    img = cv2.imread(f)
    face_arr = asarray(img)
    faces.append(face_arr)

  samples = asarray(faces,'float32')

  samples = preprocess_input(samples,version=2)

  model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

  yhat = model.predict(samples)

  return yhat

In [0]:
face_list = []
for elem in os.listdir('./Images'):
  face_list.append('./Images/' + elem)
y_train_encoder = get_embeddings(face_list)
x_train_decoder = get_embeddings(face_list)














94699520/94694792 [==============================] - 1s 0us/step


In [0]:
y_train_encoder.shape

(81, 2048)

In [0]:
# Using y_train to create decoder

In [0]:
# Input is y_train and output is images from Images folder

In [0]:
# Augmenting Audio

In [0]:
# shutil.rmtree('Processed_Audio')
os.mkdir('Processed_Audio')
temp_dict = dict_audio.copy()
for elem in os.listdir('./Audio'):
  sound = AudioSegment.from_mp3('./Audio/' + elem)
  sound_new = sound
  while temp_dict[elem[:-4]] < 6:
    print("here")
    sound_new += sound
    temp = temp_dict[elem[:-4]]
    temp_dict[elem[:-4]] = 2*temp
    sound = sound_new

  extract = sound[0:6*1000]

  extract.export('Processed_Audio/' + elem, format="mp3")


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [0]:
# # to_remove = ['2f32XSMYlDk']
# for elem in to_remove:
#   os.remove('Processed_Audio/' + elem + '.mp3')

In [0]:
# Preparing x_train

In [0]:
# shutil.rmtree('Spectograms')
os.mkdir('Spectograms')
for filename in os.listdir("./Processed_Audio"):
  audio_path = './Processed_Audio/' + filename
  x , sr = librosa.load(audio_path,sr = 15925)
  print(audio_path)
  print(type(x), type(sr))
  print(x.shape, sr)
  ipd.Audio(audio_path)
  window_size = 25
  window = np.hanning(window_size)
  h1 = 160
  print(h1)

  stft  = librosa.core.spectrum.stft(x, n_fft=512, hop_length=h1,win_length=window_size, window=window)
  out = 2 * np.abs(stft) / np.sum(window_size)
  with open('./Spectograms/' + filename[:-4] + '.pkl','wb') as f:
    pickle.dump(stft,f)
  
  # librosa.display.waveplot(x, sr=sr)
  # mfccs = librosa.feature.mfcc(x, sr=sr)
  # print(mfccs.shape)
  # librosa.display.specshow(mfccs, sr=sr, x_axis='time')
  # print()




./Processed_Audio/512K2S3De-A.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/G7xm-5aDZyg.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/YXHQsV27oO0.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/M_HcFLa0b1Q.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/8BQxktcqJjU.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/ipcZj9l2a6o.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/Wl3HSpsiIb4.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/JjduaMIoKvI.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/a2iQ7kB5b6s.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/sujFCXbYkMo.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/uxCc_3ybp1w.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160

In [40]:
stft.shape

(257, 598)

In [0]:
# for elem in stft:
#   print(elem)

In [0]:
# Todo: For all pickle files in spectogram 

In [0]:
def get_encoder_network_input(k):
  shape_x = k.shape[0]
  shape_y = k.shape[1]
  mod = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      mod[i][j] = np.abs(k[i][j])

  theta_ = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      theta_[i][j] = np.angle(k[i][j])

  real = np.zeros((shape_x,shape_y))
  complex_ = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      temp = mod[i][j]
      real[i][j] = temp * math.cos(complex_[i][j])
      complex_[i][j] = temp * math.sin(complex_[i][j])

  real = np.expand_dims(real,axis = 0)
  complex_ = np.expand_dims(complex_,axis = 0)
  combined = np.concatenate((real,complex_))

  return combined

In [42]:
encoder_input = []
for elem in os.listdir('./Spectograms/'):
  with open('./Spectograms/' + elem,'rb') as f:
    print('./Spectograms/' + elem)
    k = pickle.load(f)
    print(k.shape)

  encoder_input.append(np.transpose(get_encoder_network_input(k)))

./Spectograms/UVDnhj-jZl0.pkl
(257, 598)
./Spectograms/ibPOxQ7XYPk.pkl
(257, 598)
./Spectograms/byZdbbWenGs.pkl
(257, 598)
./Spectograms/QoQF8N5ZsQA.pkl
(257, 598)
./Spectograms/vNs_u_S8pmw.pkl
(257, 598)
./Spectograms/qpEzCs23PWE.pkl
(257, 598)
./Spectograms/5qy9Ujv9XdM.pkl
(257, 598)
./Spectograms/U_AVbzYkQ3E.pkl
(257, 598)
./Spectograms/xwxbJkXRJHw.pkl
(257, 598)
./Spectograms/UBL2Vowiulk.pkl
(257, 598)
./Spectograms/8nQBG5hvjpk.pkl
(257, 598)
./Spectograms/8BQxktcqJjU.pkl
(257, 598)
./Spectograms/oX2WY-uO3tM.pkl
(257, 598)
./Spectograms/AvWWVOgaMlk.pkl
(257, 598)
./Spectograms/qzM4wshoqGs.pkl
(257, 598)
./Spectograms/MZnQ3eZuUAE.pkl
(257, 598)
./Spectograms/GtM2sM6r3So.pkl
(257, 598)
./Spectograms/nC3xThn3sBI.pkl
(257, 598)
./Spectograms/uxCc_3ybp1w.pkl
(257, 598)
./Spectograms/-A9gdf3j2xo.pkl
(257, 598)
./Spectograms/awqKt7frvJI.pkl
(257, 598)
./Spectograms/JjduaMIoKvI.pkl
(257, 598)
./Spectograms/Y8HMIm8mdns.pkl
(257, 598)
./Spectograms/2f32XSMYlDk.pkl
(257, 598)
./Spectograms/QY

In [43]:
len(encoder_input)

81

In [0]:
# For encoder

# Input : encoder input as np array
# Output : y_train_encoder

# FOr decoder

# Input : y_pred_encoder
# Output : Images from Images folder

In [0]:
x_enc_train = np.asarray(encoder_input)

In [45]:
x_enc_train.shape

(81, 598, 257, 2)

In [46]:
y_train_encoder.shape

(81, 2048)

In [0]:
with open(PATH_TO_MAIN + 'Pickles/encoder_trainX_81.pkl','wb') as f:
    pickle.dump(x_enc_train,f)

In [0]:
with open(PATH_TO_MAIN + 'Pickles/encoder_trainY_81.pkl','wb') as f:
    pickle.dump(y_train_encoder,f)

In [53]:
from google.colab import files
files.download(PATH_TO_MAIN + 'Pickles/encoder_trainX_81.pkl')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 51406, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
!nvidia-smi